In [2]:

import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.svm import SVC
from itertools import product
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV,StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.ensemble import (RandomForestClassifier,RandomForestRegressor,GradientBoostingClassifier,GradientBoostingRegressor,BaggingClassifier,VotingClassifier,AdaBoostClassifier)
from sklearn.naive_bayes import GaussianNB
from sklearn.externals.six import StringIO
#from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
#from imblearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,StandardScaler
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,roc_auc_score,
                             f1_score,accuracy_score,confusion_matrix,mean_absolute_error,r2_score,mean_squared_error)
import datetime

import time
import os
import sys

from scipy import misc
from scipy.stats import randint


In [3]:
df=pd.read_csv('./santandar/train.csv')

In [4]:
df.describe()
df.columns

Index([u'ID_code', u'target', u'var_0', u'var_1', u'var_2', u'var_3', u'var_4',
       u'var_5', u'var_6', u'var_7',
       ...
       u'var_190', u'var_191', u'var_192', u'var_193', u'var_194', u'var_195',
       u'var_196', u'var_197', u'var_198', u'var_199'],
      dtype='object', length=202)

In [9]:
def get_score(clf, train_features, train_labels):
    X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.4, random_state=0)
    clf.fit(X_train, y_train)
    print clf.score(X_test, y_test)

def get_accuracy(clf, train_features, train_labels):
    scores = cross_validation.cross_val_score(clf, train_features, train_labels, cv=10)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

def random_searchSVM(train_features, train_labels, n_jobs,objective):
    param_dist = {
#       {'C': [1, 10, 100], 'kernel': ['linear']},
       'C': [1,10,50,100], 'gamma': [0.0001, 0.001], 'kernel': ['rbf','linear']}
#          {'C': [1], 'gamma': [0.001], 'kernel': ['rbf']}
    

    clf = GridSearchCV(svm.SVC(class_weight="balanced"), param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_

def random_searchGB(train_features, train_labels, n_jobs,objective):
    param_dist = {'n_estimators': randint(5, 300),
                   'min_samples_leaf': randint(5, 30),
                   'max_depth': randint(5, 10),
                   'learning_rate': [0.05,0.001]
                  }

    clf = RandomizedSearchCV(GradientBoostingClassifier(),param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_

def random_searchRF(train_features, train_labels, n_jobs,objective):
    param_dist = {
        'n_estimators'      : randint(5, 300),
        'max_features'      : randint(3, 20),
        'min_samples_split' : randint(3, 100),
        'max_depth'         : randint(3, 100)}

    clf = RandomizedSearchCV(RandomForestClassifier(class_weight="balanced"), param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_

def random_searchVC(train_features,train_labels,n_jobs,objective):
    clf1 = LogisticRegression(random_state=1,class_weight="balanced")
    clf2 = RandomForestClassifier(random_state=1,class_weight="balanced")
    clf3 = GaussianNB()
    eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='soft')
    params = {'lr__C': randint(1, 300), 'rf__n_estimators': randint(5, 300),'rf__max_features':randint(3, 20),'rf__min_samples_split' : randint(3, 100),'rf__max_depth':randint(3, 100)}
    clf = RandomizedSearchCV(estimator=eclf,param_distributions=params,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_

def random_searchAB(train_features,train_labels,n_jobs,objective):
    param_dist = {
        'algorithm'      : ['SAMME', 'SAMME.R'],
        'n_estimators'      : randint(5, 300)}
    bdt = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1))
    clf = RandomizedSearchCV(estimator=bdt,param_distributions=param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_ 

def random_searchNC(train_features,train_labels,n_jobs,objective):
    param_dist = {
        'activation'      : ['identity', 'logistic', 'tanh', 'relu'],
        'hidden_layer_sizes' : [(5,2), (10,5),(10,20,5),(5,10,10,5)],
        'alpha' : [1e-5,1e-4,1e-3],
        'solver' : ['lbfgs', 'sgd', 'adam']}
      
    clf = GridSearchCV(estimator= MLPClassifier(),param_grid=param_dist,n_jobs=n_jobs,scoring=objective)
    clf.fit(train_features, train_labels)
    return clf.best_estimator_,clf.best_score_,clf.best_params_  

def best_estimater(train_features,train_labels,test_features,test_labels,objective):
    start_time = time.time()
    n_jobs = 30

    #objective = 'f1' or 'accuracy' or 'recall' or 'roc_auc'
    a_RF,b_RF,c_RF=random_searchRF(train_features, train_labels,n_jobs,objective)
  
    print("---RF finished %s seconds ---" % round(time.time() - start_time,2))
    duration_time = time.time()
  
    a_GB,b_GB,c_GB=random_searchGB(train_features, train_labels,n_jobs,objective)
  
    print("---GB finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    a_SVM,b_SVM,c_SVM=random_searchSVM(train_features, train_labels,n_jobs,objective)
  
    print("---SVM finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    a_VC,b_VC,c_VC=random_searchVC(train_features, train_labels,n_jobs,objective)
  
    print("---VotingClassification finished %s seconds ---" % round(time.time() - start_time,2))
    duration_time = time.time()
  
    a_AB,b_AB,c_AB=random_searchAB(train_features, train_labels,n_jobs,objective)
  
    print("---AdaBoost finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    a_NC,b_NC,c_NC=random_searchNC(train_features, train_labels,n_jobs,objective)
  
    print("---NeuralNetwork finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
  
  
    RF_ML = a_RF
    RF_ML.fit(train_features,train_labels)
    RF_ML.score(test_features,test_labels)
    RF_y_pred = RF_ML.predict(test_features)
  
  
    print("---RF fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    GB_ML = a_GB
    GB_ML.fit(train_features,train_labels)
    GB_ML.score(test_features,test_labels)
    GB_y_pred = GB_ML.predict(test_features)
  
  
    print("---GB fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    SVM_ML = a_SVM
    SVM_ML.fit(train_features,train_labels)
    SVM_ML.score(test_features,test_labels)
    SVM_y_pred = SVM_ML.predict(test_features)
  
  
    print("---SVM fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
  
    VC_ML = a_VC
    VC_ML.fit(train_features,train_labels)
    VC_ML.score(test_features,test_labels)
    VC_y_pred = VC_ML.predict(test_features)
  
  
    print("---VotingClassification fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    AB_ML = a_AB
    AB_ML.fit(train_features,train_labels)
    AB_ML.score(test_features,test_labels)
    AB_y_pred = AB_ML.predict(test_features)
  
  
    print("---AdaBoosting fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    NC_ML = a_NC
    NC_ML.fit(train_features,train_labels)
    NC_ML.score(test_features,test_labels)
    NC_y_pred = NC_ML.predict(test_features)
  
  
    print("---Neural Network fitting finished %s seconds ---" % round(time.time() - duration_time,2))
    duration_time = time.time()
  
    if objective == "f1":
        RF_score=f1_score(test_labels, RF_y_pred)
        GB_score=f1_score(test_labels, GB_y_pred)
        SVM_score=f1_score(test_labels, SVM_y_pred)
        VC_score=f1_score(test_labels, VC_y_pred)
        NC_score=f1_score(test_labels, NC_y_pred)
        AB_score=f1_score(test_labels, AB_y_pred)
    elif objective == "recall":
        RF_score=recall_score(test_labels, RF_y_pred)
        GB_score=recall_score(test_labels, GB_y_pred)
        SVM_score=recall_score(test_labels, SVM_y_pred)
        VC_score=recall_score(test_labels, VC_y_pred)
        NC_score=recall_score(test_labels, NC_y_pred)
        AB_score=recall_score(test_labels, AB_y_pred)
    elif objective == "accuracy":
        RF_score=accuracy_score(test_labels, RF_y_pred)
        GB_score=accuracy_score(test_labels, GB_y_pred)
        SVM_score=accuracy_score(test_labels, SVM_y_pred)
        VC_score=accuracy_score(test_labels, VC_y_pred)
        NC_score=accuracy_score(test_labels, NC_y_pred)
        AB_score=accuracy_score(test_labels, AB_y_pred)
    elif objective == "roc_auc":
        RF_score=roc_auc_score(test_labels, RF_y_pred)
        GB_score=roc_auc_score(test_labels, GB_y_pred)
        SVM_score=roc_auc_score(test_labels, SVM_y_pred)
        VC_score=roc_auc_score(test_labels, VC_y_pred)
        NC_score=roc_auc_score(test_labels, NC_y_pred)
        AB_score=roc_auc_score(test_labels, AB_y_pred)
    print 'RF:'+str(RF_score)+', GB:'+str(GB_score)+', SVM:'+str(SVM_score)+', VC:'+str(VC_score)+', AB:'+str(AB_score)+', NC:'+str(NC_score)
    score_list = [RF_score,GB_score,SVM_score,VC_score,NC_score,AB_score]
    if RF_score ==max(score_list):
        return a_RF,b_RF,c_RF
    elif GB_score ==max(score_list):
        return a_GB,b_GB,c_GB
    elif SVM_score ==max(score_list):
        return a_SVM,b_SVM,c_SVM
    elif VC_score ==max(score_list):
        return a_VC,b_VC,c_VC
    elif NC_score ==max(score_list):
        return a_NC,b_NC,c_NC
    elif AB_score ==max(score_list):
        return a_AB,b_AB,c_AB
    else:
        print('The error happens')


        

In [10]:
train_features1=df["target"]
train_labels1=df.drop('ID_code',axis=1)
train_labels1=train_labels1.drop('target',axis=1)

In [ ]:
y_train, y_test, X_train, X_test = train_test_split(train_features1, train_labels1,
                                                    test_size=0.96, 
                                                    random_state=1)

In [ ]:
a,b,c=best_estimater(X_train, y_train,X_test,y_test,'accuracy')

---RF finished 22.65 seconds ---
---GB finished 130.76 seconds ---


In [ ]:
a_txt = str(a)
print(a_txt)
print(b)
print(c)

In [ ]:
ML = a
test_s2 = 0.5
X_train, X_test, y_train, y_test = train_test_split(train_features2, train_labels2,
                                                    test_size=test_s2,       
                                                    random_state=0)

ML.fit(X_train, y_train)
ML.score(X_test,y_test)
y_pred = ML.predict(X_test)
print("accuracy="+str(accuracy_score(y_test, y_pred)))
#print(average_precision_score(y_test,predict_y,average='weighted'))
print("f1="+str(f1_score(y_test, y_pred)))
print("precision="+str(precision_score(y_test, y_pred)))
print("recall="+str(recall_score(y_test, y_pred)))
#Vertucal:actual/ Horizontal:prediction
confusion_matrix(y_test,y_pred)

In [ ]:
ML2=a
ML2.fit(train_labels1,train_features1)

df_test=pd.read_csv('./santandar/test.csv')
df_label_test=df_test.drop('ID_code',axis=1)
y_pred=ML2.predict(df_label_test)
df_answer=pd.concat([df_test["ID_code"],y_pred],axis=1)
df_answer.to_csv('./santandar/submission.csv')